#                              OPEN AI

In [1]:
# command to install the openai Library
! pip install openai

### To get the list of files.

In [5]:
import openai

# use the key to access the specific open ai account
openai.api_key = ' '

In [6]:
# store the file list in  a response variable
response = openai.File.list()

response

<OpenAIObject list at 0x7f11bc795e90> JSON: {
  "data": [
    {
      "bytes": 1073,
      "created_at": 1627093182,
      "filename": "sample.jsonl",
      "id": "file-CHHNbL5BScrs4tJfmuuWS84y",
      "object": "file",
      "purpose": "search",
      "status": "processed",
      "status_details": null
    },
    {
      "bytes": 1613,
      "created_at": 1627093797,
      "filename": "sample.jsonl",
      "id": "file-bLsoZuJD2pmSOZEkq7BHwPc3",
      "object": "file",
      "purpose": "search",
      "status": "processed",
      "status_details": null
    },
    {
      "bytes": 1661,
      "created_at": 1627371558,
      "filename": "sample.jsonl",
      "id": "file-IlR8hf1u1JeAGSYyeH3rEUce",
      "object": "file",
      "purpose": "search",
      "status": "processed",
      "status_details": null
    },
    {
      "bytes": 6141,
      "created_at": 1627643469,
      "filename": "ngfm json clean.jsonl",
      "id": "file-omUysquh6clJtnFjXXgZMpOB",
      "object": "file",
      "pu

## Convert the json file to a proper jsonl format:

In [ ]:
# convert the json to jsonl - instead of ',' we have a new line for each json object
# difference between json and jsonl is the delimiter
# remove some of the double quotes which might lead to error

In [ ]:
# open the file to read the lines in the json
with open('C:/Users/nikhi/Documents/rmit/4th sem/lexata/post type training/ngfm json clean.jsonl') as f:
    lines = f.readlines()

# create a json 
create_json_list = [] # list which will store the json

for i in range(len(lines)):
    initial_str = ''
    
    if i == 0:
        initial_str = lines[i][0:36]
    else:
        initial_str = lines[i][0:35]
    for j in lines[i][35:(len(lines[i])-4)]:    
        j = j.replace('"','')
        initial_str = initial_str + j
    initial_str = initial_str + '"}'
    create_json_list.append(initial_str)

textfile = open("final ngfm.jsonl", "w")
for element in create_json_list:
    textfile.write(element + "\n")
textfile.close()

# Upload the jsonl file to the open ai

In [ ]:
response = openai.File.create(file=open("final ngfm.jsonl"), purpose="search")

print(response)
# note the id of file

Upload progress: 100%|███████████████████████████████████████████████████████████| 86.7k/86.7k [00:00<00:00, 89.5kit/s]


{
  "bytes": 86408,
  "created_at": 1627679978,
  "filename": "final ngfm.jsonl",
  "id": "file-q6sYJFu6AoTVHA71qhoBIFIm",
  "object": "file",
  "purpose": "search",
  "status": "uploaded",
  "status_details": null
}


# Semantic Search Endpoint:

In [7]:

search_response = openai.Engine("davinci").search(
    search_model="davinci",
    query="Which documents are available to the public?",
    max_rerank=5,
    file="file-q6sYJFu6AoTVHA71qhoBIFIm",
    return_metadata=True
)

print(search_response)

{
  "data": [
    {
      "document": 0,
      "metadata": "11599###ngfm",
      "object": "search_result",
      "score": 134.342,
      "text": "Interpretation of made available to the public and filed, delivered or submitted: Documents made available to the public include not only information filed on SEDAR but also information on a website and disclosure provided through social media platforms.The Instrument uses the terms filed and submitted. This Policy also uses the term delivered. Material that is filed in a jurisdiction will be made available to the public in that jurisdiction, subject to the provisions of securities legislation in the local jurisdiction. Material that is delivered to a regulator or securities regulatory authority, or submitted to a recognized exchange, but not filed, is not generally required under securities legislation to be made available to the public."
    },
    {
      "document": 1,
      "metadata": "11523###ngfm",
      "object": "search_result",
  

In [ ]:
# to access the specific documents from the list
search_response.data[3].text

"Incorporating information by reference: (1) Subject to subsections (3) and (4), an issuer may incorporate by reference the information required under any of the following provisions, if the reference is to the issuer's MD&amp;A: (a) subparagraph 6(1)(e)(ii) ; (b) paragraph 7(2)(d) ; (c) subparagraph 8(c)(iii) ; (d) paragraph 9(c) ; (e) subparagraph 10(1)(b)(ii) ; (f) paragraph 11(b) . (2) If, as permitted under subsection (1), an issuer incorporates required information by reference into a document, the issuer must include all of the following in the document: (a) a statement indicating that the information is incorporated by reference; (b) a statement that specifies the location of the information in the MD&amp;A; (c) a statement that the MD&amp;A is available on SEDAR at www.sedar.com. (3) Despite subsection (1), an issuer must not incorporate by reference the information referred to in subsection (1) in its MD&amp;A if the document that contains the specified financial measure is a

In [8]:
# Text Summarization

In [ ]:
# according to the specification of openai add tl;dr at the end of the text so the model understands that it has to complete
# the remaining text or in our case summarise the text

# create the text variable with the text and the tldr tag
text_to_summarise = search_response.data[3].text + "\n tl;dr:"


summary_response = openai.Completion.create(engine="davinci", prompt = text_to_summarise, temperature = 0.3,
                        max_tokens=140, top_p=1, frequency_penalty=0,
                        presence_penalty=0,stop=["\n"])


In [ ]:
summary_response

<OpenAIObject text_completion id=cmpl-3RXqE9KUYkMqvahs46gOBrA6PYjFn at 0x284cace7180> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " If you're going to present non-GAAP metrics, you need to disclose the GAAP metrics too."
    }
  ],
  "created": 1627693954,
  "id": "cmpl-3RXqE9KUYkMqvahs46gOBrA6PYjFn",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}